# XGBoost
Reference: https://www.datacamp.com/tutorial/xgboost-in-python

In [ ]:
import seaborn as sns

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import xgboost as xgb

from sklearn.metrics import mean_squared_error

import warnings




warnings.filterwarnings("ignore")




ratings = pd.read_csv("_.csv")

ratings.head()

In [ ]:
ratings.shape

In [ ]:
ratings.describe()

In [ ]:
ratings.describe(exclude=np.number)

In [5]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays

X, y = ratings.drop('User_Rating', axis=1), ratings[['User_Rating']]

# print(X)
# print(y)

In [ ]:
# Extract text features

cats = X.select_dtypes(exclude=np.number).columns.tolist()

print(cats)

print(X)

# Convert to Pandas category

for col in cats:

   X[col] = X[col].astype('category')

# print(X)

In [ ]:
X.dtypes

In [8]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# Create regression matrices

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)

dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [10]:
# Define hyperparameters

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [11]:


evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]

n = 10000

model = xgb.train(

   params=params,

   dtrain=dtrain_reg,

   num_boost_round=n,

   evals=evals,

   verbose_eval=100,

   early_stopping_rounds=25

)

In [12]:
# testing it on unseen data
preds = model.predict(dtest_reg)

In [ ]:
rmse = mean_squared_error(y_test, preds, squared=False)


print(f"RMSE of the base model: {rmse:.3f}")

In [20]:
# Cross-Validation. Once satisfied with its score, you must retrain it on the full data before deployment.

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 1000

results = xgb.cv(

   params, dtrain_reg,

   num_boost_round=n,

   nfold=5,

   early_stopping_rounds=20

)

In [ ]:
results.head()

In [ ]:
best_rmse = results['test-rmse-mean'].min()

best_rmse